In [1]:
from google.colab import drive 
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
%cd /content/drive/My Drive
!ls

/content/drive/My Drive
'Colab Notebooks'	  gan_plot.png	      'NN Structure'
 discriminator_plot.png   generator_plot.png   Portrait


In [3]:
!pip install mtcnn

In [4]:
from mtcnn.mtcnn import MTCNN
from numpy import asarray
from PIL import Image
import numpy as np
import os

In [5]:
# load images in a directory
def load_image():
    images = list()
    dir = "Portrait/subPortrait"
    # enumerate files
    count = 0
    for filename in os.listdir(dir):
          paint = Image.open(dir + '/' + filename)
          paint = paint.convert('RGB')
          pixel = np.asarray(paint)
          images.append(pixel)
          count += 1
          if count == 3:
            break
    return np.asarray(images)

In [6]:
# Faces extraction
def face_extra(pixel_list, required_size = (80, 80)):
    model = MTCNN()
    face_list = []
    count = 0
    for pixel in pixel_list:
        # Detect face in the image
        face = model.detect_faces(pixel)
        # Skip cases where we could not detect a face
        if len(face) == 0:
            continue
        # extract details of the face
        x1, y1, width, height = face[0]['box']
        # force detected pixel values to be positive (bug fix)
        x1, y1 = abs(x1), abs(y1)
        # convert into coordinates
        x2, y2 = x1 + width, y1 + height
        # retrieve face pixels
        face_pixels = pixel[y1:y2, x1:x2]
        # resize pixels to the model size
        image = Image.fromarray(face_pixels)
        image = image.resize(required_size)
        face_array = asarray(image)
        face_list.append(face_array)
        count += 1
        if count ==3:
          break
    return np.asarray(face_list)

In [7]:
pixels = load_image()
face = face_extra(pixels)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  from ipykernel import kernelapp as app


In [ ]:
np.savez_compressed("Portrait/portrait.npz", face)